In [2]:
import os
import pandas as pd
import json
import ast

current_dir = os.getcwd()
root_dir = os.path.dirname(os.path.dirname(current_dir))
_RAWFIRESPATH = f"{root_dir}/RawData/Historical_FiresRAW"

/tmp/ipykernel_14063/2194959734.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
_year = 2005
_file = f"{_RAWFIRESPATH}/{_year}/2005-01-01_37.7388357_-8.750062888101874.csv"
df = pd.read_csv(f"{_file}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names = df.columns.tolist()
print(updated_column_names)

_file_radiation = _file.replace(".csv", "_radiation.csv")
df = pd.read_csv(f"{_file_radiation}")
print(df.head())

df = pd.read_csv(f"{_file_radiation}")
# print(df.head())
column_names = df.columns.tolist()
print(column_names)

# Get all column names
column_names = df.columns.tolist()

# Find columns that contain 'hourly_units'
columns_to_remove = [col for col in column_names if 'hourly_units' in col]
columns_to_remove.extend(['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly.time', 'hourly.global_tilted_irradiance_instant', 'hourly.diffuse_radiation_instant', 'hourly.terrestrial_radiation_instant', 'hourly.direct_radiation_instant', 'hourly.shortwave_radiation_instant', 'hourly.direct_normal_irradiance_instant'])

# Remove the columns
df = df.drop(columns=columns_to_remove)

# Print the updated column names
updated_column_names_radiation = df.columns.tolist()
print(updated_column_names_radiation)

# Convert the lists to sets
set1 = set(updated_column_names_radiation)
set2 = set(updated_column_names)

# Find the common elements
common_elements = set1 & set2

# Print the common elements
print(common_elements)

['latitude', 'longitude', 'generationtime_ms', 'utc_offset_seconds', 'timezone', 'timezone_abbreviation', 'elevation', 'hourly_units.time', 'hourly_units.temperature_2m', 'hourly_units.relative_humidity_2m', 'hourly_units.dew_point_2m', 'hourly_units.apparent_temperature', 'hourly_units.precipitation', 'hourly_units.rain', 'hourly_units.snowfall', 'hourly_units.snow_depth', 'hourly_units.weather_code', 'hourly_units.pressure_msl', 'hourly_units.surface_pressure', 'hourly_units.cloud_cover', 'hourly_units.cloud_cover_low', 'hourly_units.cloud_cover_mid', 'hourly_units.cloud_cover_high', 'hourly_units.et0_fao_evapotranspiration', 'hourly_units.vapour_pressure_deficit', 'hourly_units.wind_speed_10m', 'hourly_units.wind_speed_100m', 'hourly_units.wind_direction_10m', 'hourly_units.wind_direction_100m', 'hourly_units.wind_gusts_10m', 'hourly_units.soil_temperature_0_to_7cm', 'hourly_units.soil_temperature_7_to_28cm', 'hourly_units.soil_temperature_28_to_100cm', 'hourly_units.soil_temperatur

In [8]:
header = ['year', 'date', 'district', 'municipality', 'parish', 'local', 'latitude', 'longitude', 'cause']

columns = header + updated_column_names + updated_column_names_radiation


df = pd.DataFrame(columns=columns)

df.to_csv('b2020.csv', index=False)


df.to_csv('b2020y.csv', index=False)

In [ ]:
_year = 2022

df = pd.read_csv("FireDataset.csv")

print(df.head())

dfsourcex = pd.read_csv(f'{_RAWFIRESPATH}/filtered_icnf_2022.csv')
filtered_df = dfsourcex[(dfsourcex['TIPO'] == 'Florestal') & (dfsourcex['TIPOCAUSA'].isin(['Desconhecida', 'Natural']))]
print(len(filtered_df))

second_row = filtered_df.iloc[0]
DIA = second_row['DIA']
MES = second_row['MES']
HORA = second_row['HORA']
LAT = second_row['LAT']
LON = second_row['LON']
ANO = second_row['ANO']
DISTRICTO = second_row['DISTRITO']
CONCELHO = second_row['CONCELHO']
FREGUESIA = second_row['FREGUESIA']

MeteorologyFile = False
RadiationFile = False

print(DIA, MES, HORA, LAT, LON, ANO)

MES = "0" + str(MES) if MES < 10 else str(MES)
DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

_fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
    dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

    print(dfMet['elevation'].iloc[0])

    list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

    print(list_from_string)

    list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

    print(list_from_string)

    new_data = {
        'year': ANO,
        'date': f"{ANO}-{MES}-{DIA}",
        'district': DISTRICTO,
        'municipality': CONCELHO,
        'parish': FREGUESIA,
        'latitude': LAT,
        'longitude': LON,
        'elevation': dfMet['elevation'].iloc[0]
    }

    try:
        for i in range(1, len(updated_column_names)):
            new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
    except Exception as e:
        print("->", e)

    try:
        for i in range(0, len(updated_column_names_radiation)):
            new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
    except Exception as e:
        print("-x", e)

    new_df = pd.DataFrame(new_data, index=[0])

    new_df.to_csv('FireDataset2022.csv', mode='a', header=False, index=False)


In [7]:
_year = 2020

df = pd.read_csv(f"b{_year}.csv")

dfsourcex = pd.read_csv(f'NaturalFires/v2filtered_icnf_{_year}.csv')


for r in range(0, len(dfsourcex)):

    row = dfsourcex.iloc[r]
    DIA = row['DIA']
    MES = row['MES']
    HORA = row['HORA']
    LAT = row['LAT']
    LON = row['LON']
    ANO = row['ANO']
    DISTRICTO = row['DISTRITO']
    CONCELHO = row['CONCELHO']
    FREGUESIA = row['FREGUESIA']
    CAUSA = row['TIPOCAUSA']
    LOCAL = row['LOCAL']

    MeteorologyFile = False
    RadiationFile = False

    if pd.isna(LOCAL):
        LOCAL = 'NC'

    if pd.isna(CAUSA):
        CAUSA = 'NC'

    print(DIA, MES, HORA, LAT, LON, ANO)

    MES = "0" + str(MES) if MES < 10 else str(MES)
    DIA = "0" + str(DIA) if DIA < 10 else str(DIA)

    _fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}.csv.csv"

    _fileName_radiation = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation.csv"

    print(_fileName, _fileName_radiation)

    isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
    if(isExistMeteorology):
        MeteorologyFile = True
    else:
        row_df = pd.DataFrame(row).transpose()  # Convert row (Series) to DataFrame
        row_df.to_csv(f'error_rows{_year}.csv', mode='a', header=False, index=False)
        
        continue

    isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
    if(isExistRadiation):
        RadiationFile = True
    else:
        row_df = pd.DataFrame(row).transpose()  # Convert row (Series) to DataFrame
        row_df.to_csv(f'error_rows{_year}.csv', mode='a', header=False, index=False)
        
        continue

    print(isExistMeteorology, RadiationFile)


    if(MeteorologyFile and RadiationFile):
        try:
            dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
            dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")
        except Exception as e:
            row_df = pd.DataFrame(row).transpose()  # Convert row (Series) to DataFrame
            row_df.to_csv(f'error_rows{_year}.csv', mode='a', header=False, index=False)
            
            continue

        try:
            print(dfMet['elevation'].iloc[0])
        except Exception as e:
            row_df = pd.DataFrame(row).transpose()  # Convert row (Series) to DataFrame
            row_df.to_csv(f'error_rows{_year}.csv', mode='a', header=False, index=False)
            
            continue


        list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

        print(list_from_string)

        list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

        print(list_from_string)

        new_data = {
            'year': ANO,
            'date': f"{ANO}-{MES}-{DIA}",
            'district': DISTRICTO,
            'municipality': CONCELHO,
            'parish': FREGUESIA,
            'local': LOCAL,
            'latitude': LAT,
            'longitude': LON,
            'cause': CAUSA,
            'elevation': dfMet['elevation'].iloc[0]
        }

        try:
            for i in range(1, len(updated_column_names)):
                new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
        except Exception as e:
            print("->", e)

        try:
            for i in range(0, len(updated_column_names_radiation)):
                new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
        except Exception as e:
            print("-x", e)

        new_df = pd.DataFrame(new_data, index=[0])

        new_df.to_csv(f'b{_year}.csv', mode='a', header=False, index=False)
       

23 6 16 37.1172222222222 -8.30944444444444 2021
23_06_16_37.1172222222222_-8.30944444444444.csv.csv 23_06_16_37.1172222222222_-8.30944444444444_radiation.csv
True True
21.0
21.6
2021-06-23T16:00
25 6 13 37.0766666666667 -8.03916666666667 2021
25_06_13_37.0766666666667_-8.03916666666667.csv.csv 25_06_13_37.0766666666667_-8.03916666666667_radiation.csv
True True
50.0
26.4
2021-06-25T13:00
28 6 18 37.3036111111111 -7.49166666666667 2021
28_06_18_37.3036111111111_-7.49166666666667.csv.csv 28_06_18_37.3036111111111_-7.49166666666667_radiation.csv
True True
121.0
29.1
2021-06-28T18:00
2 7 13 37.0961111111111 -8.26861111111111 2021
02_07_13_37.0961111111111_-8.26861111111111.csv.csv 02_07_13_37.0961111111111_-8.26861111111111_radiation.csv
True True
106.0
19.7
2021-07-02T13:00
3 7 16 37.1355555555556 -8.01111111111111 2021
03_07_16_37.1355555555556_-8.01111111111111.csv.csv 03_07_16_37.1355555555556_-8.01111111111111_radiation.csv
True True
192.0
32.4
2021-07-03T16:00
4 7 16 37.1369444444444 

In [ ]:
print(list_from_string)

In [ ]:
with open('missing_files.txt', 'r') as f:
    mf = f.readlines()
    

print(mf)
dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip()}")
print(dfMet.head())

dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip().replace('.csv.csv', '_radiation.csv')}")


In [ ]:
import requests
_year = 2023
print(mf[2].strip().replace('.csv.csv', ''))
gerrad = mf[2].strip().replace('.csv.csv', '').split('_')
DIA = gerrad[0]
MES = gerrad[1]
HORA = gerrad[2]
LAT = gerrad[3]
LON = gerrad[4]

print(DIA, MES, HORA, LAT, LON)

_DATA = f"{ANO}-{MES}-{DIA}"


_fileName = f"{DIA}_{MES}_{HORA}_{LAT}_{LON}_radiation"

        
isExist = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}.csv')
if(isExist):
    print(f'File already exists')
else:
    url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=shortwave_radiation,direct_radiation,diffuse_radiation,direct_normal_irradiance,global_tilted_irradiance,terrestrial_radiation,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant"
    #url = f"https://archive-api.open-meteo.com/v1/archive?latitude={LAT}&longitude={LON}&start_date={_DATA}&end_date={_DATA}&hourly=temperature_2m,relative_humidity_2m,dew_point_2m,apparent_temperature,precipitation,rain,snowfall,snow_depth,weather_code,pressure_msl,surface_pressure,cloud_cover,cloud_cover_low,cloud_cover_mid,cloud_cover_high,et0_fao_evapotranspiration,vapour_pressure_deficit,wind_speed_10m,wind_speed_100m,wind_direction_10m,wind_direction_100m,wind_gusts_10m,soil_temperature_0_to_7cm,soil_temperature_7_to_28cm,soil_temperature_28_to_100cm,soil_temperature_100_to_255cm,soil_moisture_0_to_7cm,soil_moisture_7_to_28cm,soil_moisture_28_to_100cm,soil_moisture_100_to_255cm,is_day,sunshine_duration,shortwave_radiation_instant,direct_radiation_instant,diffuse_radiation_instant,direct_normal_irradiance_instant,global_tilted_irradiance_instant,terrestrial_radiation_instant&timezone=GMT"
    response = requests.get(url)
    print(response.status_code)
    data = response.json()
    df = pd.json_normalize(data)
    # Save the DataFrame to a CSV file
    df.to_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}.csv", index=False)


In [ ]:
with open('missing_files.txt', 'r') as f:
    mf = f.readlines()
    

print(mf[0].split(",")[1])
#dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip()}")
#print(dfMet.head())

#dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{mf[2].strip().replace('.csv.csv', '_radiation.csv')}")


In [ ]:
df = pd.read_csv("FireDataset.csv")

_fileName = mf[0].split(",")[1].strip()

_fileName_radiation = mf[0].split(",")[1].replace('.csv.csv', '_radiation.csv').strip()

print(_fileName, _fileName_radiation)

isExistMeteorology = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName}')
if(isExistMeteorology):
    MeteorologyFile = True
else:
    with open('missing_files.txt', 'a') as f:
        f.write(str(r) + ',' + _fileName+ '\n')
    
    

isExistRadiation = os.path.isfile(f'{_RAWFIRESPATH}/{_year}/{_fileName_radiation}')
if(isExistRadiation):
    RadiationFile = True
else:
    with open('missing_files.txt', 'a') as f:
        f.write(str(r) + ',' + _fileName+ '\n')
    
    

print(isExistMeteorology, RadiationFile)

if(MeteorologyFile and RadiationFile):
    try:
        dfMet = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName}")
        dfRad = pd.read_csv(f"{_RAWFIRESPATH}/{_year}/{_fileName_radiation}")

        print(dfMet['elevation'].iloc[0]) # este 0 refere-se ao dfMet

        list_from_string = ast.literal_eval(dfMet[updated_column_names[2]].iloc[0])[HORA]

        print(list_from_string)

        list_from_string = ast.literal_eval(dfMet[updated_column_names[1]].iloc[0])[HORA]

        print(list_from_string)

        #FALTA CAUSE

        new_data = {
            'year': ANO,
            'date': f"{ANO}-{MES}-{DIA}",
            'district': DISTRICTO,
            'municipality': CONCELHO,
            'parish': FREGUESIA,
            'latitude': LAT,
            'longitude': LON,
            #FALTA CAUSE
            'elevation': dfMet['elevation'].iloc[0]
        }

        try:
            for i in range(1, len(updated_column_names)):
                new_data[updated_column_names[i]] = ast.literal_eval(dfMet[updated_column_names[i]].iloc[0])[HORA]
        except Exception as e:
            print(e)

        try:
            for i in range(0, len(updated_column_names_radiation)):
                new_data[updated_column_names_radiation[i]] = ast.literal_eval(dfRad[updated_column_names_radiation[i]].iloc[0])[HORA]
        except Exception as e:
            print(e)

        new_df = pd.DataFrame(new_data, index=[0])
        new_df.to_csv('FireDataset.csv', mode='a', header=False, index=False)
        print("aqui")
    except Exception as e:
        print(e)
        with open('missing_files.txt', 'a') as f:
            f.write(str(r) + ',' + _fileName+ '\n')